# Find Spatial Locations
## Buffer a point, return all data within spatial geometry of buffer

In the following example, we will buffer a point on the map and return all features within the buffer.

-----

**Directions:** Begin by importing all the necessary libraries.

**Note:** You do not need to import <code>getpass</code> to provide password securely to ArcGIS. If you connect with just the GIS portal URL and your username, you will be prompted to input your password automatically.
* <code>features</code> module from <code>arcgis</code> is used for working with feature data, feature layers and collections of feature layers in the GIS. It also contains the spatial analysis functions which operate against feature data.
* <code>geometry</code> module from <code>arcgis</code> provides functions which use geometric types as input and output as well as functions for easily converting geometries between different representations.

For more information, view the documentation: [https://developers.arcgis.com/python/guide/overview-of-the-arcgis-api-for-python/]("https://developers.arcgis.com/python/guide/overview-of-the-arcgis-api-for-python/") .

In [1]:
# import libraries
import getpass
from arcgis.gis import *
from arcgis import geometry
from arcgis import features

In [2]:
# Utilize getpass for password management in Jupyter notebooks
password = getpass.getpass()

········


In [3]:
# Connect to GIS
gis = GIS('http://www.arcgis.com', 'alex.brown.localgovne', password)
# gis = GIS('https://neenterprise.esri.com/portal', 'abrown', password)
gis

GIS @ http://localgovne.maps.arcgis.com version:7.1

**Directions:** Manually create a point in the [NAD83 / New York Long Island (ftUS)]("https://epsg.io/2263") coordinate system, and convert the feature into a feature set (i.e., in memory) for using in geoprocessing tools.

In [16]:
# Manually create a point utilizing either decimal degrees or a projected coordinate 
# system (NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet)
# To properly buffer the boundary, need to pass in State Plane coordinates

# pt_coords = features.feature.Point({"x" : -73.989058, "y" : 40.753149, "spatialReference" : {"wkid" : 4326}})
pt_coords = features.feature.Point({"x" : 987281.588, "y" : 213669.358, "spatialReference" : {"wkid" : 2263}})
print(pt_coords)

pt = geometry.Point(pt_coords)
print(pt)
feat = features.Feature(geometry=pt, attributes={'OBJECTID': 1,
                                                        'name': 'Test_point',
                                                        'type': 'Feature'})
print(feat)

# Convert the feature into a feature set to utilize in geoprocessing tools
fset = features.FeatureSet([feat])
fset
print(fset)

{'x': 987281.588, 'y': 213669.358, 'spatialReference': {'wkid': 2263}}
{'x': 987281.588, 'y': 213669.358, 'spatialReference': {'wkid': 102718, 'latestWkid': 2263}}
{"geometry": {"x": 987281.588, "y": 213669.358, "spatialReference": {"wkid": 102718, "latestWkid": 2263}}, "attributes": {"OBJECTID": 1, "name": "Test_point", "type": "Feature"}}
{"features": [{"geometry": {"x": 987281.588, "y": 213669.358, "spatialReference": {"wkid": 102718, "latestWkid": 2263}}, "attributes": {"OBJECTID": 1, "name": "Test_point", "type": "Feature"}}], "objectIdFieldName": "OBJECTID", "spatialReference": {"wkid": 102718, "latestWkid": 2263}, "geometryType": "esriGeometryPoint", "fields": [{"name": "OBJECTID", "alias": "OBJECTID", "type": "esriFieldTypeOID", "sqlType": "sqlTypeOther"}, {"name": "name", "alias": "name", "type": "esriFieldTypeString", "sqlType": "sqlTypeOther"}, {"name": "type", "alias": "type", "type": "esriFieldTypeString", "sqlType": "sqlTypeOther"}]}


**Directions:** Search for a feature service containing 311 data. The variable <code>incidents_service</code> returns the first indexed search result. <code>incident_layers[0]</code> returns the first indexed layer from the feature service. Finally, add the result to a map.

In [17]:
# Search for NYC 311 Data, put query directly in search
crime_s = gis.content.search("title:311_Service_Requests_(Jan_2019_to_Present)", item_type="Feature Service")
incidents_service = crime_s[0]

incident_layers = incidents_service.layers
incidents = incident_layers[0]

In [18]:
# Create a map object, add layers then visualize

map_1 = gis.map("New York, NY")
map_1.add_layer(incidents)
map_1.add_layer(fset)
map_1

MapView(layout=Layout(height='400px', width='100%'))

**Directions:** Create a 500 foot buffer around the previously designated point feature set, using keeping the results in memory. Add the buffer to a map.

**Note:** Designating an output name would save the results.

In [19]:
# Buffer the point using projected coordinate system(NAD 1983 StatePlane New York Long Island US Feet)
# Do not save to permanent layer
point_buff = features.use_proximity.create_buffers(fset.to_dict(), distances=[500], units = 'Feet', 
                                                   context={"wkid" : 2263})
# This option would save the output
#point_buff = features.use_proximity.create_buffers(fset.to_dict(), distances=[500], units = 'Feet', 
#                                                   output_name='Buffer_output', context={"wkid" : 2263})

map_1.add_layer(point_buff)

**Directions:** Run the [Summarize Within]("https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.analysis.html?highlight=summarize%20within#arcgis.features.analysis.summarize_within") tool to identify the number of 311 calls within the buffered area. 

Since <code>output_name</code> in the below code is None, the results are outputed as a dictionary. 

In [60]:
# Run summarize within tool to look at how many points fall within the boundary.
output = features.analysis.summarize_within(sum_within_layer=point_buff, summary_layer=incidents, sum_shape=True, 
                                   summary_fields=[], group_by_field='text_gener', minority_majority=False, 
                                   percent_shape=False, output_name=None, context={"wkid" : 2263}, gis=None)

In [61]:
# The output of the summarize within did not specifify an output_name, the result is a dictionary.  Grab the dictionary keys
# to separate both the total numbers and grouped values.
print(output)
print(type(output))

# Return just FeatureCollection
final = output.get('result_layer')
related = output.get('group_by_summary')

{'result_layer': <FeatureCollection>, 'group_by_summary': <FeatureCollection>}
<class 'dict'>


In [62]:
# FeatureCollections are different than FeatureLayerCollections; unless one saves the layer using output_name
# a where clause cannot be utilized.
finallayers = final.query()
finallayers

<FeatureSet> 1 features

In [63]:
# Show query results in data table
finallayers.sdf

,AnalysisArea,BUFF_DIST,Join_ID,OBJECTID,ORIG_FID,Point_Count,SHAPE,name,type
0,0.072905,500,1,1,1,217,"{""rings"": [[[-75.12734031650234, 39.9826679230...",Test_point,Feature


In [64]:
# FeatureCollections are different than FeatureLayerCollections; unless one saves the layer using output_name
# a where clause cannot be utilized.
relatedlayers = related.query()
relatedlayers

<FeatureSet> 23 features

In [65]:
# Show query results in data table
relatedlayers.sdf

,Join_ID,OBJECTID,Point_Count,objectid_1,text_gener
0,1,1,2,1,Aggravated Assault Firearm
1,1,2,6,2,Aggravated Assault No Firearm
2,1,3,42,3,All Other Offenses
3,1,4,1,4,Arson
4,1,5,3,5,Burglary Non-Residential
5,1,6,15,6,Burglary Residential
6,1,7,5,7,Disorderly Conduct
7,1,8,10,8,DRIVING UNDER THE INFLUENCE
8,1,9,2,9,Forgery and Counterfeiting
9,1,10,10,10,Fraud


In [54]:
# Create a second map object for clarity
map_2 = gis.map("New York, NY")
map_2.add_layer(finallayers)
map_2

MapView(layout=Layout(height='400px', width='100%'))